# Transfer Learning VGG16+Xception

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
import keras

#from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
#import pylab
#plt.rcParams['figure.figsize'] = 10, 10
#%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../ShipIceberg/Data"]).decode("utf8"))

Using TensorFlow backend.


sample_submission.csv
test.json
train.json



In [2]:
train = pd.read_json("../ShipIceberg/Data/train.json")
target_train=train['is_iceberg']

# 一直cannot resolve memory block
test = pd.read_json("../ShipIceberg/Data/test.json")

In [3]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [4]:
#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

X_band_3 = np.zeros((1604,75,75))
for i in range(0,1604):
    subt = abs(X_band_1[i]-X_band_2[i])
    W1 = subt/subt.max()
    W2=1-W1
    X_band_3[i]=W1 * X_band_1[i]+W2 * X_band_2[i]
    
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])

X_band_test_3 = np.zeros((8424,75,75))
for i in range(0,8424):
    subt = abs(X_band_test_1[i]-X_band_test_2[i])
    W1 = subt/subt.max()
    W2=1-W1
    X_band_test_3[i]=W1 * X_band_test_1[i]+W2 * X_band_test_2[i]
    


In [5]:
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)

X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

# del train;del test;

In [6]:
#Import Keras.
#from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator
batch_size=64

In [7]:
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   #es = EarlyStopping('val_loss', patience=10, mode="min")
   es = EarlyStopping('val_loss', patience=20, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]

In [11]:
def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
#     x = base_model.get_layer('block5_pool').output
    x= base_model.output
    x = GlobalMaxPooling2D()(x)
    base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

#     base_model2 = Xception(weights='imagenet', include_top=False, input_tensor = base_model.input,
#                  input_shape=X_train.shape[1:], classes=1)
    x2 = base_model2.output
    x2 = GlobalAveragePooling2D()(x2)

    merge_one = concatenate([x, x2, angle_layer])
    
    #     merge_one = x
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
#     merge_one = Dense(1024, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one) # 参数原来0.3
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
#     merge_one = Dense(256, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)

#     merge_one = Dropout(0.6)(merge_one)
#     predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

In [9]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "./model/%s_aug_vgg_xception_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=10)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
#                 steps_per_epoch=24,
                steps_per_epoch=len(X_train_cv)//batch_size,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [12]:
preds=myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0


/home/jayden/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
16/16 [==============================] - 33s - loss: 0.7439 - acc: 0.5215 - val_loss: 1.1872 - val_acc: 0.5308
Epoch 2/100
16/16 [==============================] - 10s - loss: 0.6813 - acc: 0.5935 - val_loss: 1.0257 - val_acc: 0.5308
Epoch 3/100
16/16 [==============================] - 10s - loss: 0.6403 - acc: 0.6088 - val_loss: 0.8350 - val_acc: 0.5103
Epoch 4/100
16/16 [==============================] - 10s - loss: 0.6095 - acc: 0.6456 - val_loss: 0.7295 - val_acc: 0.4841
Epoch 5/100
16/16 [==============================] - 10s - loss: 0.5770 - acc: 0.6794 - val_loss: 0.7116 - val_acc: 0.5720
Epoch 6/100
16/16 [==============================] - 10s - loss: 0.5369 - acc: 0.7253 - val_loss: 0.7066 - val_acc: 0.5794
Epoch 7/100
16/16 [==============================] - 10s - loss: 0.5382 - acc: 0.7326 - val_loss: 0.6910 - val_acc: 0.5944
Epoch 8/100
16/16 [==============================] - 9s - loss: 0.4821 - acc: 0.7539 - val_loss: 0.7453 - val_acc: 0.6019
Epoch 9/100
16/16

16/16 [==============================] - 9s - loss: 0.1544 - acc: 0.9501 - val_loss: 0.4598 - val_acc: 0.8299
Epoch 68/100
16/16 [==============================] - 9s - loss: 0.1886 - acc: 0.9200 - val_loss: 0.4379 - val_acc: 0.8542
Epoch 69/100
16/16 [==============================] - 9s - loss: 0.1842 - acc: 0.9160 - val_loss: 0.2604 - val_acc: 0.8953
Epoch 70/100
16/16 [==============================] - 9s - loss: 0.1968 - acc: 0.9110 - val_loss: 0.2450 - val_acc: 0.8935
Epoch 71/100
16/16 [==============================] - 9s - loss: 0.1828 - acc: 0.9308 - val_loss: 0.2442 - val_acc: 0.8916
Train loss: 0.158227715289
Train accuracy: 0.932647334849
Test loss: 0.22073347925
Test accuracy: 0.897196263019

===================FOLD= 1
Epoch 1/100
16/16 [==============================] - 31s - loss: 0.7225 - acc: 0.5273 - val_loss: 1.5243 - val_acc: 0.5308
Epoch 2/100
16/16 [==============================] - 10s - loss: 0.6611 - acc: 0.6027 - val_loss: 1.3337 - val_acc: 0.5308
Epoch 3/100

Test loss: 0.218782997326
Test accuracy: 0.914018692034

===================FOLD= 2
Epoch 1/100
16/16 [==============================] - 36s - loss: 0.7515 - acc: 0.5322 - val_loss: 0.6649 - val_acc: 0.5225
Epoch 2/100
16/16 [==============================] - 8s - loss: 0.6905 - acc: 0.5563 - val_loss: 0.7199 - val_acc: 0.5300
Epoch 3/100
16/16 [==============================] - 8s - loss: 0.5971 - acc: 0.6594 - val_loss: 0.8037 - val_acc: 0.4981
Epoch 4/100
16/16 [==============================] - 8s - loss: 0.6255 - acc: 0.6487 - val_loss: 0.7103 - val_acc: 0.5150
Epoch 5/100
16/16 [==============================] - 8s - loss: 0.5283 - acc: 0.7220 - val_loss: 0.7755 - val_acc: 0.5449
Epoch 6/100
16/16 [==============================] - 9s - loss: 0.5386 - acc: 0.7331 - val_loss: 0.7423 - val_acc: 0.5599
Epoch 7/100
16/16 [==============================] - 8s - loss: 0.4656 - acc: 0.7735 - val_loss: 0.7670 - val_acc: 0.5787
Epoch 8/100
16/16 [==============================] - 8s - los

16/16 [==============================] - 9s - loss: 0.1391 - acc: 0.9524 - val_loss: 0.3219 - val_acc: 0.8670
Epoch 67/100
16/16 [==============================] - 9s - loss: 0.1850 - acc: 0.9246 - val_loss: 0.3108 - val_acc: 0.8708
Epoch 68/100
16/16 [==============================] - 9s - loss: 0.1655 - acc: 0.9372 - val_loss: 0.3359 - val_acc: 0.8689
Epoch 69/100
16/16 [==============================] - 9s - loss: 0.1779 - acc: 0.9238 - val_loss: 0.3246 - val_acc: 0.8727
Epoch 70/100
16/16 [==============================] - 9s - loss: 0.1635 - acc: 0.9393 - val_loss: 0.3477 - val_acc: 0.8614
Epoch 71/100
16/16 [==============================] - 9s - loss: 0.1747 - acc: 0.9288 - val_loss: 0.3183 - val_acc: 0.8708
Epoch 72/100
16/16 [==============================] - 9s - loss: 0.1902 - acc: 0.9245 - val_loss: 0.2979 - val_acc: 0.8783
Epoch 73/100
16/16 [==============================] - 9s - loss: 0.1783 - acc: 0.9307 - val_loss: 0.3055 - val_acc: 0.8633
Epoch 74/100
16/16 [=========

In [ ]:
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('./model/subXception+Mobile.csv', index=False)
# submission.to_csv('subVGG+Mobile.csv', index=False)
# submission.to_csv('subVGG+Xception.csv', index=False)